In [52]:
import numpy as np
import cv2
import os
import numpy as np
import tensorflow as tf
import time
import PIL.Image as Image
import pickle

In [147]:
def crop_cells(image_name,image,bboxes,classes,out_dir = "./",cell_size = 64,cell_propotion = 0.8):
    if os.path.exists(out_dir) == False:
        os.makedirs(out_dir)
    num_boxes = bboxes
    image_h, image_w, color = image.shape
    for i in range(num_boxes.shape[0]):
        start_x = int(num_boxes[i][0] * image_w)
        start_y = int(num_boxes[i][1] * image_h)
        cell_w = int(num_boxes[i][2] * image_w)
        cell_h = int(num_boxes[i][3] * image_h)
        # check the cell size, if the cell is too small or too big, skip it
        if cell_w < (cell_size*cell_propotion) or cell_h < (cell_size*cell_propotion):
            # print("cell too small",cell_w,cell_h)
            continue
        elif cell_w > (cell_size/cell_propotion) or cell_h > (cell_size/cell_propotion):
            # print("cell too big",cell_w,cell_h)
            continue
        elif num_boxes[i][5]<0.5:
            continue
        #check the cell type by column 4, if the directory does not exist, create one
        if os.path.exists(out_dir+f"{classes[int(num_boxes[i][4])]}") == False:
            os.makedirs(out_dir+f"{classes[int(num_boxes[i][4])]}")
        # crop the cell depends on the larger edge
        if cell_w > cell_h:
            sub_image = image[start_y-cell_w//2:start_y+cell_w//2 , start_x-cell_w//2:start_x+cell_w//2]
        else:
            sub_image = image[start_y-cell_h//2:start_y+cell_h//2 , start_x-cell_h//2:start_x+cell_h//2]
        # resize the cell to 64*64
        sub_image = cv2.resize(sub_image,(cell_size,cell_size))
        # save the cell image
        cv2.imwrite(f"{out_dir+classes[int(num_boxes[i][4])]}/{image_name}_{i+1}.png", sub_image)


In [148]:
image_name = "A3_31145"
image = cv2.imread('../HCT_region_select/output/A3/ROI/{}.png'.format(image_name))
pred_bboxes = np.array(
[[ 0.18152618 , 0.457537   , 0.11544328 , 0.15253988 , 1. , 0.99530786]
 ,[ 0.87146711 , 0.51213038 , 0.13695805 , 0.13692804 , 1. , 0.79374796]
 ,[ 0.04332939 , 0.40949506 , 0.08531047 , 0.13185708 , 1. , 0.75805676]
 ,[ 0.61988497 , 0.42331761 , 0.11087529 , 0.12813179 , 2. , 0.99717295]
 ,[ 0.770841   , 0.79351085 , 0.13573016 , 0.15782709 , 2. , 0.86535299]
 ,[ 0.81683069 , 0.89062089 , 0.12654251 , 0.13377713 , 2. , 0.83696181]
 ,[ 0.80842119 , 0.97140604 , 0.13412331 , 0.05361312 , 2. , 0.73328191]
 ,[ 0.27493244 , 0.91747075 , 0.13962418 , 0.14723885 , 2. , 0.72571719]
 ,[ 0.27416682 , 0.92291272 , 0.1464784  , 0.1397194  , 5. , 0.58951086]
 ,[ 0.11493276 , 0.65372229 , 0.19815867 , 0.2301493  ,14. , 0.46205008]])
classes_file = "./model/obj.names"
with open(classes_file, 'r') as f:
    classes = f.read().splitlines()
crop_cells(image_name,image, pred_bboxes,classes,"./{}/".format(image_name))
# check if the pickle file exists, if not, create one
if os.path.exists("./{}.pkl".format(image_name.split("_")[0])) == False:
    #create pickle file
    with open("./{}.pkl".format(image_name.split("_")[0]), 'wb') as f:
        my_dict = {image_name:pred_bboxes}
        pickle.dump(my_dict, f)
else:
    # load the pickle file
    with open("./{}.pkl".format(image_name.split("_")[0]), 'rb') as f:
        my_dict = pickle.load(f)
    # add the new image to the dictionary
    my_dict[image_name] = pred_bboxes
    # save the dictionary to the pickle file
    with open("./{}.pkl".format(image_name.split("_")[0]), 'wb') as f:
        pickle.dump(my_dict, f)



10
cell too small
43 67
cell too small
68 27
cell too big
101 117


In [155]:
#load the bboxes
with open("/home/weber50432/AML_image_processing/HCT_cell_detection/output/A26/A26.pkl", 'rb') as f:
    bboxes = pickle.load(f)
print(len(bboxes))

4080


In [145]:
classes_file = "./model/obj.names"

with open(classes_file, 'r') as f:
    classes = f.read().splitlines()
print(classes)
print(len(classes))


['Neutrophil', 'Metamyelocyte', 'Myelocyte', 'Promyelocyte', 'Blast', 'Erythroblast', 'Megakaryocyte_nucleus ', 'Lymphocyte', 'Monocyte', 'Plasma_cell', 'Eosinophil', 'Basophil', 'Megakaryocyte', 'Other_cell', 'Debris', 'Histiocyte', 'Platelet', 'Platelet_clump', 'Mast_cell']
19


In [4]:
patch_path = "/home/weber50432/AML_image_processing/HCT_cell_detection/output/A143_15038/A143_15038_1.png"
img =Image.open(patch_path).convert('RGB')
img = img.resize((224,224),Image.BILINEAR)
img.save(patch_path)

In [9]:
slide_num = "A3"
slide_path = f"/home/exon_storage1/aml_slide/ROI_level0_pixel512/{slide_num}/"
if os.path.exists(f"./output/{slide_num}") == False:
    os.makedirs(f"./output/{slide_num}")
start_time = time.time()
for i in os.listdir(slide_path):
  if os.path.exists(slide_path+i) == True:
    i = i.split(".")[0]
    print(i)

A3_22549
A3_39077
A3_48834
A3_36916
A3_48497
A3_17142
A3_17523
A3_20053
A3_20395
A3_13917
A3_23280
A3_23984
A3_32950
A3_52482
A3_16469
A3_28319
A3_53565
A3_30425
A3_30857
A3_22185
A3_50306
A3_15733
A3_32228
A3_17875
A3_29401
A3_38713
A3_08497
A3_53562
A3_52485
A3_21427
A3_24718
A3_12111
A3_17139
A3_30422
A3_50301
A3_27968
A3_15734
A3_32588
A3_29711
A3_15355
A3_29406
A3_14263
A3_38714
A3_28627
A3_29067
A3_47417
A3_21829
A3_40159
A3_49201
A3_28703
A3_48833
A3_17145
A3_24012
A3_17524
A3_20054
A3_13562
A3_20392
A3_32632
A3_23287
A3_30167
A3_26508
A3_42335
A3_23637
A3_28310
A3_16812
A3_21429
A3_27262
A3_48841
A3_24716
A3_27603
A3_16777
A3_17137
A3_27966
A3_26881
A3_32586
A3_13510
A3_23645
A3_47419
A3_28629
A3_23300
A3_40157
A3_21827
A3_43423
A3_49579
A3_21484
A3_28684
A3_18972
A3_12828
A3_06016
A3_26506
A3_23258
A3_25072
A3_15030
A3_23639
A3_14670
A3_26167
A3_31561
A3_36918
A3_32302
A3_21483
A3_30791
A3_35251
A3_15741
A3_29764
A3_52118
A3_28683
A3_47775
A3_19347
A3_25075
A3_26160
A3_18610
A

In [11]:
slides_path = "/home/exon_storage1/aml_slide/ROI_level0_pixel512/"
for slide_num in os.listdir(slides_path):
    if os.path.exists(f"./output/{slide_num}") == True:
        print(f"{slide_num} already exists")
    else:
        print(f"Creating {slide_num}")

Creating A60
Creating A607
Creating A670
Creating A799
Creating A777
Creating A347
Creating A330
Creating A579
Creating A597
Creating A403
Creating A474
Creating A1023
Creating A1159
Creating A149
A143 already exists
Creating A134
Creating A490
Creating A824
Creating A409
Creating A1029
Creating A1153
Creating A1124
Creating A642
Creating A635
Creating A738
Creating A25
Creating A1215
Creating A205
Creating A308
Creating A648
Creating A732
Creating A745
Creating A1066
Creating A966
Creating A446
Creating A988
Creating A431
Creating A1116
Creating A1082
Creating A1161
Creating A536
Creating A816
Creating A982
Creating A106
Creating A171
Creating A323
Creating A764
Creating A669
Creating A719
Creating A614
Creating A224
Creating A837
Creating A560
Creating A1140
Creating A150
Creating A410
Creating A467
Creating A1030
Creating A1047
Creating A721
Creating A756
Creating A311
Creating A366
Creating A382
Creating A651
Creating A626
Creating A162
Creating A1172
Creating A428
Creating A872
Cr